In [ ]:
!pip install crewai

In [ ]:
!pip install crewai crewai-tools

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# **IMPORTS**

In [ ]:
import os
from crewai import Agent, Task, Crew
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  PDFSearchTool,
  SerperDevTool
)
from crewai import LLM

In [ ]:

os.environ["SERPER_API_KEY"] = "API_KEy"
os.environ['MISTRAL_API_KEY'] = "API_KEY"


# **TOOLS for the CREW**

# **Semantic Search on Resume Using Mistral AI and Google Embeddings**

This parts is use to perform a semantic search on a resume PDF using the Mistral AI model for language understanding and Google embeddings for document retrieval. The configuration ensures that we can effectively match job descriptions with relevant parts of the resume.

In [ ]:
# Initialize the semantic search tool for resume search using Mistral AI and Google embeddings
semantic_search_resume = PDFSearchTool(
    config=dict(
        # Configure the language model to use Mistral AI as the provider
        llm=dict(
            provider="mistralai",  # Change to use Mistral AI as the provider
            config=dict(
                model="mistral-7B",  # Specify the Mistral AI model to use
            ),
        ),
        # Configure the embedding model to use Google's document retrieval embedding model
        embedder=dict(
            provider="google",  # You can change this to other providers like openai, ollama, etc.
            config=dict(
                model="models/embedding-001",  # Specify the embedding model for retrieval tasks
                task_type="retrieval_document",  # Define the task type as document retrieval
            ),
        ),
    ),
    # Specify the path to the resume PDF for semantic search
    mdx="Your_pdf.pdf"  # Path to the resume PDF file
)

# Semantic Search with SerperDev, Web Scraping, and Resume File Reading

This part demonstrates the use of multiple tools for performing semantic search, web scraping, and reading a resume PDF. The SerperDev tool is used for search queries, the ScrapeWebsite tool extracts information from web pages, and the FileRead tool is used to read the resume for analysis.


In [ ]:
# Initialize the search tool using SerperDev for semantic search capabilities
search_tool = SerperDevTool(
      config=dict(
        engine="google_scholar",  # Use Google Scholar for research-oriented searches
        num_results=10,          # Get more results
    )
)

# Initialize the web scraping tool to extract content from websites
scrape_tool = ScrapeWebsiteTool()

# Initialize the file reading tool to read the resume PDF file for further analysis
read_resume = FileReadTool(file_path="path.pdf")  # Path to the resume PDF file

# **Configuring LLMs with Mistral and Gemini Models for Language Tasks**

This section demonstrates how to configure two separate LLM instances using the Mistral and Gemini models. These LLMs will be used for different natural language processing tasks, such as semantic search or content generation. The parameters like model temperature and API keys are customizable based on the user's needs.

In [ ]:
mistral = LLM(
    model="mistral-small-2402",  # Specify the Mistral model to use
    temperature=0.1,  # Adjust the creativity level of the model's responses
    base_url="https://api.mistral.ai/v1",  # Base URL for the Mistral API
    api_key="API_KEY"  # Your Mistral API key (replace as needed)
)


gemini = LLM(
    model="gemini/gemini-pro",  # Specify the Gemini model
    temperature=0.1,  # Control the level of variability in the model's output
    api_key="API_KEY"  # Your Gemini API key (replace as needed)
)


# **AGENTS**

# Agent 1: Tech Job Researcher for Job Posting Analysis

This section introduces the `Tech Job Researcher` agent, whose goal is to analyze job postings and provide valuable insights to help job applicants. The agent uses various tools, such as web scraping and search, combined with an LLM (Mistral model) to perform in-depth research and data extraction.

In [ ]:
researcher = Agent(
    role="Tech Job Researcher",  # Define the role of the agent
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",  # Set the goal for the agent
    tools=[scrape_tool, search_tool],  # Assign the necessary tools for research (scraping and search)
    verbose=True,  # Enable verbose mode to print detailed process logs
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."  # Backstory for the agent's expertise
    ),
    llm=gemini
)

# Agent 2: Personal Profiler for Engineers

This section introduces the `Personal Profiler for Engineers`, whose goal is to perform thorough research on job applicants to help them stand out in the competitive job market. The agent utilizes various tools, including scraping, resume analysis, and semantic search, and is powered by an LLM (Mistral model) to provide personalized profile-building insights.

In [ ]:
# Initialize the second agent, 'Personal Profiler for Engineers', for applicant profiling
profiler = Agent(
    role="Personal Profiler for Engineers",  # Define the agent's role as a profiler for engineers
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market",  # Set the goal to improve applicants' visibility in the job market
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],  # Assign tools for scraping, searching, and resume analysis
    verbose=True,  # Enable verbose mode for detailed output logs
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."  # Backstory describing the agent's expertise in profiling
    ),
    llm=mistral
)

# Agent 3: Resume Strategist for Engineers

This section introduces the `Resume Strategist for Engineers`, an agent whose purpose is to identify the most effective ways to make resumes stand out in the competitive job market. Using tools

In [ ]:
# Initialize the third agent, 'Resume Strategist for Engineers', for resume optimization
resume_strategist = Agent(
    role="Resume Strategist for Software Engineers",  # Define the agent's role focused on resume strategy
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",  # Set the goal to optimize resumes for better visibility
    tools=[read_resume, semantic_search_resume],  # Assign tools for resume analysis, web scraping, and job searching
    verbose=True,  # Enable verbose mode for detailed logging of the agent's actions
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."  # Backstory highlighting the agent's expertise in resume optimization
    ),
    llm=mistral
)


# Agent 4: Engineering Interview Preparer

This section introduces the `Engineering Interview Preparer`, an agent designed to generate interview questions and talking points based on a candidate's resume and job requirements. Using tools for scraping, searching, and resume analysis, this agent, powered by the Gemini model, ensures that candidates are well-prepared for job interviews.

In [ ]:
# Initialize the fourth agent, 'Engineering Interview Preparer', for interview preparation
interview_preparer = Agent(
    role="Engineering Interview Preparer",  # Define the agent's role as an interview preparer
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",  # Set the goal to help candidates prepare for interviews
    tools=[scrape_tool, search_tool, read_resume, semantic_search_resume],  # Assign tools for scraping, searching, and resume analysis
    verbose=True,  # Enable verbose mode to print detailed logs of the agent's actions
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."  # Backstory highlighting the agent's expertise in interview preparation
    ),
    llm=gemini
)


# **TASK**

# Task for Researcher Agent: Extract Job Requirements

This section demonstrates how to assign a task to the `Researcher Agent` to analyze a job posting URL and extract key job requirements, including necessary skills, qualifications, and experiences. The task uses the agent’s tools and expertise to gather and categorize this information in a structured format.

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

# Task for Profiler Agent: Compile Comprehensive Profile

This section outlines a task for the `Profiler Agent` to compile a detailed personal and professional profile using information from a GitHub URL and a personal write-up. The task involves synthesizing information from these sources to create a comprehensive profile document.


In [ ]:
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "  # Task description indicating the need for a profile compilation
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."  # Guide the agent to extract and synthesize information
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "  # Specify the expected output as a detailed profile
        "project experiences, contributions, interests, and "
        "communication style."  # Outline the key components to be included in the profile
    ),
    agent=profiler,  # Assign the Profiler Agent to complete the task
    async_execution=True  # Enable asynchronous execution to allow the agent to work in the background
)


# Task for Resume Strategist Agent: Align Resume with Job Requirements

This section describes a task for the `Resume Strategist Agent` to tailor a candidate's resume based on their profile and the job requirements extracted from previous tasks. The objective is to enhance the resume's content to better reflect the candidate’s abilities in alignment with the job posting.

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using **only** the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

# Task for Interview Preparer Agent: Develop Interview Materials

This section outlines a task for the `Interview Preparer Agent` to create a list of potential interview questions and talking points based on the tailored resume and job requirements. The goal is to help the candidate prepare effectively for their interview by focusing on relevant discussion points that align with their qualifications and the job posting.


In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a list of 10 potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


# Job Application Crew Setup

This section defines the `Job Application Crew`, which comprises multiple agents working collaboratively to assist candidates in their job application process. The crew includes a researcher, profiler, resume strategist, and interview preparer, each tasked with specific responsibilities to streamline the application journey.

In [ ]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

# **Runnig the Crew**

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://branchinternational.applytojob.com/apply/nT6J8qzm72/ML-Engineer-Intern?source=Our%20Career%20Page%20Widget',
    'github_url': 'YOUR_GITHUB_LINK',
    'personal_writeup': """Hello, my name is (name). I am a third-year engineering student at VESIT (Mumbai), majoring in Artificial Intelligence and
Data Science. Over the past two years, I've developed a strong proficiency in Full Stack Web Development, having
completed several projects that showcase my skills in building comprehensive applications. I have a solid grasp of core
Java and am well-versed in machine learning algorithms. Currently, I am actively solving LeetCode problems, with a
rating of 1550, and am expanding my knowledge in Deep Learning and Generative AI.
"""

}

In [ ]:
result = job_application_crew.kickoff(inputs=job_application_inputs)